In [1]:
import ipytest
from dataclasses import dataclass
from typing import TextIO, Generator

from collections import Counter
from collections.abc import Iterator

ipytest.autoconfig()

In [2]:
from pathlib import Path

In [3]:
data_path = Path.cwd() / 'data' / 'day5_input.txt'

In [4]:
data_path.exists()

True

In [5]:
@dataclass(frozen=True)
class PointCoordinates:
    x: int = 0
    y: int = 0

In [6]:
def row_iter(source: TextIO) -> Generator[list[str, str], None, None]:
    for line in source:
        yield line.strip('\n').split(' -> ')

In [7]:
def create_vent_lines(row_iter: Generator[list[str, str], None, None]) -> Generator[tuple[PointCoordinates, PointCoordinates], None, None]:
    for starting_coordinate, ending_coordinate in row_iter:
        yield (
            PointCoordinates(x=int(starting_coordinate.split(',')[0]), y=int(starting_coordinate.split(',')[1])),
            PointCoordinates(x=int(ending_coordinate.split(',')[0]), y=int(ending_coordinate.split(',')[1]))
        )

In [8]:
def get_rows(file_path: Path) -> Iterator[tuple[PointCoordinates, PointCoordinates]]:
    with open(file_path, 'r') as file:
        yield from create_vent_lines(row_iter(file))

In [9]:
vent_lines = list(get_rows(data_path))

In [ ]:
def generate_points_between(start: PointCoordinates, end: PointCoordinates, include_diagonals: bool = False) -> list[None | PointCoordinates]:
    """
    Generate all points between two coordinates (start and end), inclusive.

    :param start: Starting point coordinates.
    :param end: Ending point coordinates.
    :param include_diagonals: If True, include diagonal points as well.
    :return: A list of PointCoordinates representing all points between start and end.
    """
    if not include_diagonals and start.x != end.x and start.y != end.y:
        return []

    x1, y1 = start.x, start.y
    x2, y2 = end.x, end.y

    x_step = 1 if x2 > x1 else -1 if x2 < x1 else 0
    y_step = 1 if y2 > y1 else -1 if y2 < y1 else 0

    points = []
    x, y = x1, y1
    while (x, y) != (x2 + x_step, y2 + y_step):
        points.append(PointCoordinates(x=x, y=y))
        x += x_step
        y += y_step

    return points

#### Part 1

In [11]:
point_counts = Counter(
    point
    for coordinate_pairs in vent_lines
    for point in generate_points_between(coordinate_pairs[0], coordinate_pairs[1])
)


In [12]:
overlapping_points = {point: count for point, count in point_counts.items() if count >= 2}

len(overlapping_points)

7142

#### Part 2

In [13]:
point_counts = Counter(
    point
    for coordinate_pairs in vent_lines
    for point in generate_points_between(coordinate_pairs[0], coordinate_pairs[1], include_diagonals=True)
)

In [14]:
overlapping_points = {point: count for point, count in point_counts.items() if count >= 2}

len(overlapping_points)

20012